In [ ]:
import networkx as nx
from community import community_louvain
from openai import OpenAI
from pyvis.network import Network
import json
from collections import  defaultdict
from dotenv import load_dotenv
import os
# 初始化知识图谱组件
from OmniStore.chromadb_store import StoreTool
from sentence_transformers import SentenceTransformer
from KnowledgeGraphManager.KGManager import KgManager
import leidenalg
from igraph import Graph as IGraph
# 基础数据结构与算法
from collections import defaultdict, deque
import numpy as np

# 图数据操作
import networkx as nx
from community import community_louvain  # Louvain算法库

# 社区检测扩展
import leidenalg  # Leiden算法库
from igraph import Graph as IGraph  # 图结构转换

# 语义计算
from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity  # 余弦相似度计算

# 动态权重处理
from functools import lru_cache  # 路径缓存优化

load_dotenv(dotenv_path="./.env")


device = os.getenv("DEVICE")


if os.getenv("IS_USE_LOCAL") == "True":
    embeddings = SentenceTransformer(
        os.getenv("EMBEDDINGS_PATH")
    ).to(device)
else:
    # 初始化模型和组件
    embeddings = SentenceTransformer(os.getenv("EMBEDDINGS")).to(device)


# 创建两个独立的存储工具
chromadb_store = StoreTool(storage_path= os.getenv("CHROMADB_PATH"), embedding_function=embeddings)

client = OpenAI(
    api_key=os.getenv("API_KEY"),
    base_url=os.getenv("BASE_URL")
)

# 多模态模型
vl_client = OpenAI(
    # 若没有配置环境变量，请用百炼API Key将下行替换为：api_key="sk-xxx"
    api_key=os.getenv("VL_API_KEY"),
    base_url=os.getenv("VL_BASE_URL")
)
from LLM.Openai_Agent import OpenaiAgent
# 创建两个独立的agent
rag_agent = OpenaiAgent(client)
kg_agent = OpenaiAgent(client)

# 创建两个独立的splitter
simple_files = os.getenv("SIMPLE", "").split(",")
semantic_files = os.getenv("SEMANTIC", "").split(",")
character_files = os.getenv("CHARACTER", "").split(",")

# 初始化默认分割器
kg_splitter = None

# 创建默认分割器
if len(simple_files) > 0:
    from TextSlicer.SimpleTextSplitter import SimpleTextSplitter
    kg_splitter = SimpleTextSplitter(2045, 1024)
elif len(semantic_files) > 0:
    from TextSlicer.SemanticTextSplitter import SemanticTextSplitter
    kg_splitter = SemanticTextSplitter(2045, 1024)
elif len(character_files) > 0:
    from TextSlicer.CharacterTextSplitter import CharacterTextSplitter
    kg_splitter = CharacterTextSplitter(separator="</end>", keep_separator=False, max_tokens=2045, min_tokens=1024)

# 创建两个独立的kg_manager
kg_manager = KgManager(agent=kg_agent, splitter=kg_splitter, embedding_model=embeddings, store=chromadb_store)

kg_manager.load_store("知识融合")


G = kg_manager.current_G
G2 = kg_manager.current_G
print(G)

# ================== 社区划分算法部分 ==================
# Louvain算法
community_map = community_louvain.best_partition(G2.to_undirected())
print(community_map, 2)

# Leiden算法
# 将networkx图转换为igraph图
nodes = list(G2.nodes())
node_id_map = {node: i for i, node in enumerate(nodes)}
edges = [(node_id_map[u], node_id_map[v]) for u, v in G2.edges()]

# 创建igraph图对象
igraph_G = IGraph(directed=False)
igraph_G.add_vertices(len(nodes))
igraph_G.add_edges(edges)

# 进行Leiden社区检测
partition = leidenalg.find_partition(
    igraph_G,
    leidenalg.ModularityVertexPartition,
    n_iterations=-1  # 使用无限迭代直到收敛
)
leiden_communities = partition.membership
leiden_community_map = {node: leiden_communities[node_id_map[node]] for node in nodes}


# ================== 可视化部分 ==================
def visualize_communities(graph, community_dict, filename):
    """通用社区可视化函数"""
    net = Network(height="600px", width="100%", notebook=True)

    # 添加节点
    for node in graph.nodes():
        net.add_node(
            node,
            label=str(node),
            group=community_dict[node],
            color="#%06x" % (community_dict[node] * 0x0F0F0F)
        )

    # 添加边
    for edge in graph.edges():
        net.add_edge(edge[0], edge[1])

    # 配置物理引擎
    net.toggle_physics(True)
    net.show(filename)


class CommunityRetriever:
    def __init__(self, graph, community_map):
        """
        :param graph: NetworkX 图对象（带权重）
        :param community_map: 社区划分字典 {节点: 社区ID}
        """
        self.graph = graph
        self.community_map = community_map
        self._build_community_index()

    def _build_community_index(self):
        """构建社区反向索引"""
        self.community_index = defaultdict(list)
        for node, comm_id in self.community_map.items():
            self.community_index[comm_id].append(node)

    def find_related_entities(self, query_entities, top_n=20):
        """
        带权重的社区实体检索
        :param query_entities: 查询实体列表
        :param top_n: 返回每个实体的相关实体数量
        :return: 排序后的相关实体字典
        """
        results = {}

        for entity in query_entities:
            if entity not in self.graph:
                continue

            # 获取实体所属社区
            comm_id = self.community_map.get(entity, -1)
            community_nodes = self.community_index.get(comm_id, [])

            # 计算相关性分数
            scores = []
            for node in community_nodes:
                if node == entity:
                    continue

                # 计算路径权重（考虑直接连接和多跳关系）
                try:
                    # 获取最短路径
                    path = nx.shortest_path(self.graph, source=entity, target=node)
                    # 计算路径权重总和
                    path_weight = sum(
                        self.graph[path[i]][path[i + 1]].get('weight', 1)
                        for i in range(len(path) - 1)
                    )
                    # 标准化处理：路径权重均值
                    score = path_weight / len(path)
                except nx.NetworkXNoPath:
                    score = 0

                scores.append((node, score))

            # 按分数降序排序
            sorted_entities = sorted(scores, key=lambda x: x[1], reverse=True)
            results[entity] = {
                'community': comm_id,
                'related': sorted_entities[:top_n]
            }

        return results

    def enhanced_community_search(
            self,
            file,
            entity_names,
            weight_threshold=0.3,
            top_n=20,
            max_hops=3,
            decay_factor=0.6,
            semantic_weight=0.4,
            community_boost=1.2
    ):
        """
        增强型社区知识检索（支持多跳关系、社区增强和语义融合）

        Args:
            file: 文件名
            entity_names: 输入实体列表
            weight_threshold: 综合权重阈值，默认0.3
            top_n: 最大返回数量，默认20
            max_hops: 最大关系跳数，默认3
            decay_factor: 多跳衰减系数，默认0.6
            semantic_weight: 语义相似度权重，默认0.4
            community_boost: 社区增强系数，默认1.2

        Returns:
            知识库条目列表（按综合得分排序）
        """
        current_G = self.get_G(file)
        if current_G is None:
            print(f"无法获取知识图谱数据: {file}")
            return []

        # 阶段1：社区检测
        partition = community_louvain.best_partition(current_G.to_undirected())
        community_ids = {partition[e] for e in entity_names if e in partition}

        # 阶段2：构建社区子图
        community_nodes = [n for n, cid in partition.items() if cid in community_ids]
        subgraph = current_G.subgraph(community_nodes)

        # 阶段3：多跳关系挖掘（带权重衰减）
        path_scores = defaultdict(float)
        for source in entity_names:
            if source not in subgraph:
                continue

            # 带衰减的广度优先搜索
            queue = deque([(source, 1.0, 0)])
            visited = defaultdict(float)

            while queue:
                node, score, hops = queue.popleft()
                if hops > max_hops:
                    continue

                # 更新路径得分（取最大值）
                path_scores[node] = max(path_scores[node], score)

                # 遍历邻居节点
                for neighbor in subgraph[node]:
                    edge_weight = subgraph[node][neighbor].get('weight', 0.5)
                    new_score = score * edge_weight * (decay_factor ** hops)

                    if new_score > visited[neighbor]:
                        visited[neighbor] = new_score
                        queue.append((neighbor, new_score, hops + 1))

        # 阶段4：社区中心性计算
        pagerank_scores = nx.pagerank(subgraph, weight='weight')

        # 阶段5：语义相似度计算
        question_text = " ".join(entity_names)
        question_embedding = self.embedding.encode(question_text)
        semantic_scores = {
            node: cosine_similarity(
                question_embedding,
                subgraph.nodes[node].get('embedding', np.zeros_like(question_embedding))
            )
            for node in path_scores.keys()
        }

        # 阶段6：动态权重融合
        combined_scores = {}
        for node in path_scores.keys():
            # 基础得分组合（路径+语义）
            base_score = (
                    (1 - semantic_weight) * path_scores[node] +
                    semantic_weight * semantic_scores[node]
            )

            # 社区增强
            if node in community_nodes:
                base_score *= community_boost

            # 中心性增强
            combined_scores[node] = (
                    0.7 * base_score +
                    0.3 * pagerank_scores.get(node, 0)
            )

        # 阶段7：边信息收集与筛选
        knowledge_base = []
        processed_pairs = set()

        # 按综合得分排序节点
        sorted_nodes = sorted(combined_scores.items(), key=lambda x: -x[1])

        for node, score in sorted_nodes[:top_n * 2]:  # 扩大候选池
            if score < weight_threshold:
                continue

            # 收集所有相关边
            for neighbor in subgraph[node]:
                edge_data = subgraph[node][neighbor]
                pair = tuple(sorted([node, neighbor]))

                if pair in processed_pairs:
                    continue

                processed_pairs.add(pair)

                # 计算边综合得分
                edge_score = (
                        0.6 * score +
                        0.3 * combined_scores.get(neighbor, 0) +
                        0.1 * edge_data.get('weight', 0.5)
                )

                if edge_score >= weight_threshold:
                    knowledge_base.append({
                        'source': node,
                        'target': neighbor,
                        'relation': edge_data.get('label', '未知关系'),
                        'context': edge_data.get('title', '无上下文'),
                        'score': round(edge_score, 3),
                        'metadata': {
                            'path_score': round(path_scores[node], 3),
                            'semantic_score': round(semantic_scores[node], 3),
                            'pagerank': round(pagerank_scores.get(node, 0), 3)
                        }
                    })

        # 最终排序和截断
        knowledge_base.sort(key=lambda x: -x['score'])
        return knowledge_base[:top_n]


# 生成Louvain可视化
visualize_communities(G, community_map, "louvain_community.html")

# 生成Leiden可视化
visualize_communities(G, leiden_community_map, "leiden_community.html")

# 初始化检索器（使用Louvain社区划分）
louvain_retriever = CommunityRetriever(G.to_undirected(), community_map)

# 初始化检索器（使用Leiden社区划分）
leiden_retriever = CommunityRetriever(G.to_undirected(), leiden_community_map)

# 示例使用
if __name__ == "__main__":
    # 假设用户问题中的实体列表
    query_entities = ["牛顿力学", "星海科技","经典物理"]

    # 使用Louvain算法的检索结果
    louvain_result = louvain_retriever.find_related_entities(query_entities)
    print("Louvain社区检索结果：")
    print(json.dumps(louvain_result, indent=5, ensure_ascii=False))

    # 使用Leiden算法的检索结果
    leiden_result = leiden_retriever.find_related_entities(query_entities)
    print("\nLeiden社区检索结果：")
    print(json.dumps(leiden_result, indent=5, ensure_ascii=False))




阶段1：模块度优化（节点分配）
初始化：每个节点独立为一个社区。
局部贪婪搜索：
遍历节点：依次将每个节点尝试移动到其邻居所在的社区。
计算增益ΔQ：评估移动后的模块度变化。公式简化为：
## 模块度增益公式（ΔQ）

ΔQ的计算公式为：
$$
\Delta Q = \frac{k_{i,\text{in}}}{2m} - \frac{\Sigma_{\text{tot}}^c \cdot k_i}{(2m)^2}
$$

### 符号说明：
- $k_{i,\text{in}}$：节点$i$与其目标社区$c$之间的边权重和。
- $\Sigma_{\text{tot}}^c$：社区$c$的总边权重（包含内外边）。
- $m$：网络总边权重（$m = \frac{1}{2}\sum A_{ij}$）。

接受移动：若ΔQ>0，则将节点移入增益最大的社区。
迭代调整：重复遍历所有节点直到模块度不再提升。
阶段2：网络聚合（构建超级节点）
合并社区：将同一社区内的节点合并为一个超级节点。
重构网络：
超级节点之间的边权重：原网络跨社区边的权重之和。
超级节点内部边权重：原社区内部边的权重之和（保留自环）。
循环迭代：将聚合后的新网络作为输入，重复阶段1和阶段2，直到模块度无法进一步优化。

模块度Q：衡量社区内实际连边与随机连边的差值之和，归一化后范围在[−0.5,1)，值越大表示社区划分越优。

一、Louvain算法的局限性
不连通社区：Louvain可能生成内部不连通的社区。
局部最优陷阱：贪婪策略易陷入次优解。
结果不稳定：节点遍历顺序影响最终划分。
小社区遗漏：分辨率限制导致无法检测微小社区。
二、Leiden算法的核心改进
1. 三阶段迭代优化
阶段1：局部节点移动（类似Louvain）。
阶段2：子社区细化（Refinement）强制社区连通。
阶段3：网络聚合（确保聚合后社区结构稳定）。
2. 允许暂时性质量下降
引入概率接受策略，以一定概率接受降低模块度的移动，逃离局部最优。
3. 精细化社区划分
子分区细化：将大社区递归拆分为连通子社区。
随机性控制：通过固定随机种子提升结果可重复性。

## 阶段1：局部节点移动
初始化：每个节点独立为社区。
遍历节点：
尝试将节点移动到邻居社区，计算模块度增益ΔQ。
接受条件：ΔQ > 0 或 以概率p=exp(ΔQ/T)接受（T为退火温度参数）。
迭代：重复直到无显著改进。
## 阶段2：子社区细化
连通性检验：对每个社区检测是否为连通图，若否则拆分。
递归拆分：对不连通社区继续划分子社区，直到所有子社区连通。
合并优化：评估子社区合并是否提升模块度。
## 阶段3：网络聚合
构建超级节点：将每个社区合并为超级节点。
边权重更新：
内部边权重：保留为自环。
跨社区边权重：原始网络跨社区边权重之和。
迭代：将聚合后的网络输入阶段1，直到模块度收敛。

|指标	| Louvain           | 	Leiden                    |
|---|-------------------|----------------------------|
|社区连通性	| 可能产生不连通社区	        | 强制保证社区内部连通                 |
|结果稳定性	| 高随机性依赖	           | 通过细化阶段降低随机性影响              |
|小社区检测	| 受分辨率限制	           | 支持更小社区的发现                  |
|时间复杂度	| O(n log n)	       | O(n log n) ~ O(n²)（更精确但稍慢） |
|适用网络规模	| 百万级节点	| 十万级节点（精度优先）                |